In [84]:
import pandas as pd
import numpy as np
from itertools import compress
from itertools import combinations
from scipy import stats
import math

In [75]:
def pre_processamento():
    xlsx = pd.ExcelFile('casos_corona.xlsx')
    df = pd.read_excel(xlsx, 'casos_corona')
    df = df[:300]
    df = df[['bairroPaciente','municipioPaciente','resultadoFinalExame','sexoPaciente']]
    #grouped = df.groupby(['bairroPaciente'])
    bairros = pd.unique(df['bairroPaciente'])

    clusters = []
    for bairro in bairros:
        cluster = []
        for i in range(len(df)):
            if(df.loc[i][0] == bairro):
                if(df.loc[i][2] == 'Positivo'):
                    resultado = 1
                else:
                    resultado = 0
                cluster.append([df.loc[i][0], df.loc[i][1], resultado, df.loc[i][3]])
        clusters.append(cluster)

    novo_df = []
    for cluster in clusters:
        for i in range(len(cluster)):
            novo_df.append(cluster[i])

    novo_df = pd.DataFrame(novo_df)
    novo_df.rename(columns={0: 'bairro_paciente', 1: 'municipio_paciente', 2: 'resultado_exame', 3: 'sexo_paciente'}, inplace=True)    
    return novo_df

In [128]:
df = pre_processamento()

In [129]:
df

,bairro_paciente,municipio_paciente,resultado_exame,sexo_paciente
0,VILA PERI,FORTALEZA,0,MASCULINO
1,VARJOTA,FORTALEZA,0,FEMININO
2,NOVA METROPOLE JUREMA,CAUCAIA,0,FEMININO
3,PARQUE POTIRA JUREMA,CAUCAIA,0,FEMININO
4,PINHEIRO,GUAIÚBA,0,MASCULINO
5,ALDEIA PARK,PACAJUS,0,MASCULINO
6,PRECABURA,EUSEBIO,0,MASCULINO
7,PL CAUCAIA,CAUCAIA,0,MASCULINO
8,JUREMA,CAUCAIA,0,MASCULINO
9,JUREMA,CAUCAIA,0,FEMININO


In [130]:
def ajustar(df):
    bairros = pd.unique(df['bairro_paciente'])
    casos_bairros = []
    for bairro in bairros:
        count = 0
        for i in range(len(df)):
            if(df.loc[i][0] == bairro):
                count = count + 1
        casos_bairros.append([bairro, count])
    casos_bairros = pd.DataFrame(casos_bairros)
    casos_bairros.rename(columns={0: 'bairro', 1: 'count'}, inplace=True)
    return casos_bairros

In [143]:
dataset = ajustar(df)
dataset

,bairro,count
0,VILA PERI,1
1,VARJOTA,1
2,NOVA METROPOLE JUREMA,1
3,PARQUE POTIRA JUREMA,1
4,PINHEIRO,1
5,ALDEIA PARK,1
6,PRECABURA,1
7,PL CAUCAIA,1
8,JUREMA,3
9,TABAPUA,1


In [148]:
def query_topk(dataset, k):
    copia = dataset.copy()
    output = []
    for i in range(k):
        indice = copia['count'].idxmax()
        output.append(copia.loc[indice][0])#([copia.loc[indice][0], copia.loc[indice][1]])
        copia.drop(indice, inplace=True,axis=0)
    return output

In [149]:
query_topk(dataset, 10)

['CENTRO',
 'ZONA RURAL',
 'ARATURI',
 'JUREMA',
 'MESSEJANA',
 'VIOLETE',
 'PARADA',
 'PIRAJA',
 'ANTONIO BEZERRA',
 'GIRILANDIA']

In [150]:
def score_function(registro, saida):
    if(registro[0] == saida):
        return registro[1]
    else:
        return 0

In [158]:
def sensibilidade(dataset):
    return np.max(np.array(dataset['count']))

In [159]:
sensibilidade(dataset)

56